# Load Data từ Silver Layer sang Gold Layer (Dimension & Fact Tables)

Notebook này sẽ load dữ liệu từ Silver layer và transform sang Gold layer với:
- Dimension tables có surrogate key tự động tăng
- Fact tables với các foreign keys tương ứng

## 1. Import Libraries và Khởi tạo Spark Session

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from datetime import datetime
import os

# Set AWS environment variables for MinIO
os.environ['AWS_REGION'] = 'us-east-1'
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'admin123'

# Khởi tạo Spark Session
spark = (
    SparkSession.builder
    .appName("Load_Silver_To_Gold")
    .master("spark://spark-master:7077")
    .config("spark.executor.memory", "1536m")
    .config("spark.executor.cores", "2")
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v2")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://gold/")
    .config("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key-id", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .config("spark.sql.catalog.nessie.s3.region", "us-east-1")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.region", "us-east-1")
    # Propagate environment variables to executors
    .config("spark.executorEnv.AWS_REGION", "us-east-1")
    .config("spark.executorEnv.AWS_ACCESS_KEY_ID", "admin")
    .config("spark.executorEnv.AWS_SECRET_ACCESS_KEY", "admin123")
    # ===== Sử dụng JAR files local =====
    .config("spark.jars", "/opt/spark/jars/hadoop-aws-3.3.4.jar,/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
print(" Spark Session đã được khởi tạo!")
print(f"Spark Master: {spark.sparkContext.master}")
print(f"Application ID: {spark.sparkContext.applicationId}")

## 2. Load Dimension Tables từ Silver Layer

### 2.1. Dim_Time - Tạo bảng thời gian từ dữ liệu benchmark

In [ ]:
print("Đang load Dim_Time...")
import gc

# Đọc dữ liệu từ Silver
df_benchmark = spark.table("nessie.silver_tables.benchmark")

# Tạo dim_time từ các year unique trong benchmark
df_time = df_benchmark.select("year").distinct()

# Tạo day, month từ year (giả sử ngày 1/1 của mỗi năm)
df_time = df_time.withColumn("day", lit(1)) \
    .withColumn("month", lit(1))

# Tạo timeKey tự động tăng
window_spec = Window.orderBy("year")
df_time = df_time.withColumn("timeKey", row_number().over(window_spec))

# Sắp xếp lại cột theo thứ tự
df_dim_time = df_time.select("timeKey", "day", "month", "year")

# Ghi vào Gold layer
df_dim_time.writeTo("nessie.gold_tables.dim_time") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_time.count()} dòng vào dim_time")
df_dim_time.show(10)

# Giải phóng bộ nhớ
df_benchmark.unpersist()
df_time.unpersist()
df_dim_time.unpersist()
del df_benchmark, df_time, df_dim_time
gc.collect()
print("✓ Đã giải phóng bộ nhớ")

### 2.2. Dim_Region - Bảng khu vực

In [ ]:
print("Đang load Dim_Region...")
import gc

# Đọc dữ liệu từ Silver
df_region_silver = spark.table("nessie.silver_tables.region")

# Tạo regionKey tự động tăng
window_spec = Window.orderBy("regionId")
df_dim_region = df_region_silver.withColumn("regionKey", row_number().over(window_spec))

# Sắp xếp lại cột
df_dim_region = df_dim_region.select("regionKey", "regionId", "regionName")

# Ghi vào Gold layer
df_dim_region.writeTo("nessie.gold_tables.dim_region") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_region.count()} dòng vào dim_region")
df_dim_region.show(10)

# Giải phóng bộ nhớ
df_region_silver.unpersist()
df_dim_region.unpersist()
del df_region_silver, df_dim_region
gc.collect()
print("✓ Đã giải phóng bộ nhớ")

### 2.3. Dim_School - Bảng trường đại học

In [ ]:
print("Đang load Dim_School...")
import gc

# Đọc dữ liệu từ Silver
df_school_silver = spark.table("nessie.silver_tables.school")

# Tạo schoolKey tự động tăng
window_spec = Window.orderBy("schoolId")
df_dim_school = df_school_silver.withColumn("schoolKey", row_number().over(window_spec))

# Sắp xếp lại cột
df_dim_school = df_dim_school.select("schoolKey", "schoolId", "schoolName", "province")

# Ghi vào Gold layer
df_dim_school.writeTo("nessie.gold_tables.dim_school") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_school.count()} dòng vào dim_school")
df_dim_school.show(10)

# Giải phóng bộ nhớ
df_school_silver.unpersist()
df_dim_school.unpersist()
del df_school_silver, df_dim_school
gc.collect()
print("✓ Đã giải phóng bộ nhớ")

### 2.4. Dim_Major - Bảng ngành học

In [ ]:
print("Đang load Dim_Major...")
import gc

# Đọc dữ liệu từ Silver
df_major_silver = spark.table("nessie.silver_tables.major")

# Tạo majorKey tự động tăng
window_spec = Window.orderBy("majorId")
df_dim_major = df_major_silver.withColumn("majorKey", row_number().over(window_spec))

# Sắp xếp lại cột
df_dim_major = df_dim_major.select("majorKey", "majorId", "majorName")

# Ghi vào Gold layer
df_dim_major.writeTo("nessie.gold_tables.dim_major") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_major.count()} dòng vào dim_major")
df_dim_major.show(10)

# Giải phóng bộ nhớ
df_major_silver.unpersist()
df_dim_major.unpersist()
del df_major_silver, df_dim_major
gc.collect()
print("✓ Đã giải phóng bộ nhớ")

### 2.5. Dim_Subject - Bảng môn học

In [ ]:
print("Đang load Dim_Subject...")
import gc

# Đọc dữ liệu từ Silver
df_subject_silver = spark.table("nessie.silver_tables.subject")

# Tạo subjectKey tự động tăng (sử dụng subjectId làm sort key)
window_spec = Window.orderBy("subjectId")
df_dim_subject = df_subject_silver.withColumn("subjectKey", row_number().over(window_spec))

# Sắp xếp lại cột
df_dim_subject = df_dim_subject.select("subjectKey", "subjectId", "subjectName")

# Ghi vào Gold layer
df_dim_subject.writeTo("nessie.gold_tables.dim_subject") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_subject.count()} dòng vào dim_subject")
df_dim_subject.show(10)

# Giải phóng bộ nhớ
df_subject_silver.unpersist()
df_dim_subject.unpersist()
del df_subject_silver, df_dim_subject
gc.collect()
print("✓ Đã giải phóng bộ nhớ")

### 2.6. Dim_Subject_Group - Bảng khối thi

In [ ]:
print("Đang load Dim_Subject_Group...")
import gc

# Đọc dữ liệu từ Silver
df_subject_group_silver = spark.table("nessie.silver_tables.subject_group")

# Tạo subjectGroupKey tự động tăng
window_spec = Window.orderBy("subjectGroupId")
df_dim_subject_group = df_subject_group_silver.withColumn("subjectGroupKey", row_number().over(window_spec))

# Sắp xếp lại cột
df_dim_subject_group = df_dim_subject_group.select(
    "subjectGroupKey", 
    "subjectGroupId", 
    "subjectGroupName", 
    "subjectCombination"
)

# Ghi vào Gold layer
df_dim_subject_group.writeTo("nessie.gold_tables.dim_subject_group") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_subject_group.count()} dòng vào dim_subject_group")
df_dim_subject_group.show(10)

# Giải phóng bộ nhớ
df_subject_group_silver.unpersist()
df_dim_subject_group.unpersist()
del df_subject_group_silver, df_dim_subject_group
gc.collect()
print("✓ Đã giải phóng bộ nhớ")

### 2.7. Dim_Selection_Method - Bảng phương thức xét tuyển

In [ ]:
print("Đang load Dim_Selection_Method...")
import gc

# Đọc dữ liệu từ Silver
df_selection_method_silver = spark.table("nessie.silver_tables.selection_method")

# Tạo selectionMethodKey tự động tăng
window_spec = Window.orderBy("selectionMethodId")
df_dim_selection_method = df_selection_method_silver.withColumn("selectionMethodKey", row_number().over(window_spec))

# Sắp xếp lại cột
df_dim_selection_method = df_dim_selection_method.select(
    "selectionMethodKey", 
    "selectionMethodId", 
    "selectionMethodName"
)

# Ghi vào Gold layer
df_dim_selection_method.writeTo("nessie.gold_tables.dim_selection_method") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_selection_method.count()} dòng vào dim_selection_method")
df_dim_selection_method.show(10)

# Giải phóng bộ nhớ
df_selection_method_silver.unpersist()
df_dim_selection_method.unpersist()
del df_selection_method_silver, df_dim_selection_method
gc.collect()
print("✓ Đã giải phóng bộ nhớ")

In [ ]:
print("Đang load Dim_Grading_Scale...")
import gc

# 1. Đọc dữ liệu từ Silver
df_grading_scale_silver = spark.table("nessie.silver_tables.grading_scale")

# 2. Tạo gradingScaleKey tự động tăng (surrogate key)
window_spec = Window.orderBy("gradingScaleId")

df_dim_grading_scale = df_grading_scale_silver.withColumn(
    "gradingScaleKey",
    row_number().over(window_spec)
)

# 3. Sắp xếp lại cột đúng thứ tự trong GOLD schema
df_dim_grading_scale = df_dim_grading_scale.select(
    "gradingScaleKey",
    "gradingScaleId",
    "value",
    "description"
)

# 4. Ghi vào Gold layer
df_dim_grading_scale.writeTo("nessie.gold_tables.dim_grading_scale") \
    .using("iceberg") \
    .createOrReplace()

print(f"Đã load {df_dim_grading_scale.count()} dòng vào dim_grading_scale")
df_dim_grading_scale.show(10, truncate=False)

# Giải phóng bộ nhớ
df_grading_scale_silver.unpersist()
df_dim_grading_scale.unpersist()
del df_grading_scale_silver, df_dim_grading_scale
gc.collect()
print("✓ Đã giải phóng bộ nhớ")

## 3. Load Fact Tables từ Silver Layer

### 3.1. Fact_Benchmark - Load và join với các Dimension tables

In [ ]:
print("Đang load Fact_Benchmark...")
import gc

# Đọc dữ liệu từ Silver
df_benchmark_silver = spark.table("nessie.silver_tables.benchmark")

# Đọc các dimension tables từ Gold để join
df_dim_time_gold = spark.table("nessie.gold_tables.dim_time")
df_dim_school_gold = spark.table("nessie.gold_tables.dim_school")
df_dim_major_gold = spark.table("nessie.gold_tables.dim_major")
df_dim_subject_group_gold = spark.table("nessie.gold_tables.dim_subject_group")
df_dim_selection_method_gold = spark.table("nessie.gold_tables.dim_selection_method")
df_dim_grading_scale_gold = spark.table("nessie.gold_tables.dim_grading_scale")

# Join từng bước để tránh Cartesian product
# Bước 1: Join với dim_time
df_fact = df_benchmark_silver.join(
    df_dim_time_gold, 
    df_benchmark_silver["year"] == df_dim_time_gold["year"], 
    "left"
).drop(df_dim_time_gold["year"])

# Bước 2: Join với dim_school
df_fact = df_fact.join(
    df_dim_school_gold, 
    df_fact["schoolId"] == df_dim_school_gold["schoolId"], 
    "left"
).drop(df_dim_school_gold["schoolId"])

# Bước 3: Join với dim_major
df_fact = df_fact.join(
    df_dim_major_gold, 
    df_fact["majorId"] == df_dim_major_gold["majorId"], 
    "left"
).drop(df_dim_major_gold["majorId"])

# Bước 4: Join với dim_subject_group
df_fact = df_fact.join(
    df_dim_subject_group_gold, 
    df_fact["subjectGroupId"] == df_dim_subject_group_gold["subjectGroupId"], 
    "left"
).drop(df_dim_subject_group_gold["subjectGroupId"])

# Bước 5: Join với dim_selection_method
df_fact = df_fact.join(
    df_dim_selection_method_gold, 
    df_fact["selectionMethodId"] == df_dim_selection_method_gold["selectionMethodId"], 
    "left"
).drop(df_dim_selection_method_gold["selectionMethodId"])

# Bước 6: Join với dim_grading_scale
df_fact = df_fact.join(
    df_dim_grading_scale_gold,
    df_fact["gradingScaleId"] == df_dim_grading_scale_gold["gradingScaleId"],
    "left"
).drop(df_dim_grading_scale_gold["gradingScaleId"])

print(f"Sau tất cả join: {df_fact.count():,} dòng")

# Giải phóng dimension tables
df_dim_time_gold.unpersist()
df_dim_school_gold.unpersist()
df_dim_major_gold.unpersist()
df_dim_subject_group_gold.unpersist()
df_dim_selection_method_gold.unpersist()
df_dim_grading_scale_gold.unpersist()
del df_dim_time_gold, df_dim_school_gold, df_dim_major_gold, df_dim_subject_group_gold, df_dim_selection_method_gold, df_dim_grading_scale_gold

# Tạo benchmarkKey tự động tăng
window_spec = Window.orderBy("timeKey", "schoolKey", "majorKey")
df_fact = df_fact.withColumn("benchmarkKey", row_number().over(window_spec))

# Tính điểm trung bình cho mỗi ngành theo từng phương thức
window_avg_spec = Window.partitionBy("timeKey", "schoolKey", "majorKey", "selectionMethodKey", "gradingScaleKey")
df_fact = df_fact.withColumn(
    "avgScoreByMajor",
    round(avg("score").over(window_avg_spec), 2)
)

# Tính toán yearlyScoreGap
window_year_spec = Window.partitionBy("schoolKey", "majorKey", "selectionMethodKey", "gradingScaleKey","subjectGroupKey").orderBy("timeKey")
df_fact = df_fact.withColumn(
    "yearlyScoreGap",
    coalesce(round(col("avgScoreByMajor") - lag("avgScoreByMajor", 1).over(window_year_spec),2), lit(0))
)

# Xếp hạng ngành trong trường
window_major_in_school_spec = Window.partitionBy("timeKey", "schoolKey", "selectionMethodKey", "gradingScaleKey").orderBy(desc("avgScoreByMajor"), "majorKey")
df_fact = df_fact.withColumn("rankAmongMajors", dense_rank().over(window_major_in_school_spec))

# Xếp hạng ngành so với các trường khác
window_major_across_schools_spec = Window.partitionBy("timeKey", "majorKey", "selectionMethodKey", "gradingScaleKey").orderBy(desc("avgScoreByMajor"), "schoolKey")
df_fact = df_fact.withColumn("rankAmongSchools", dense_rank().over(window_major_across_schools_spec))

# Select các cột cuối cùng
df_fact_benchmark = df_fact.select(
    "benchmarkKey",
    "subjectGroupKey",
    "timeKey",
    "majorKey",
    "schoolKey",
    "selectionMethodKey",
    "gradingScaleKey",
    col("score").cast("float"),
    col("avgScoreByMajor").cast("float"),
    col("yearlyScoreGap").cast("float"),
    "rankAmongMajors",
    "rankAmongSchools"
)

# Ghi vào Gold layer
df_fact_benchmark.writeTo("nessie.gold_tables.fact_benchmark") \
    .using("iceberg") \
    .partitionedBy("timeKey") \
    .createOrReplace()

print(f"Đã load {df_fact_benchmark.count()} dòng vào fact_benchmark")
df_fact_benchmark.show(10)

# Giải phóng bộ nhớ
df_benchmark_silver.unpersist()
df_fact.unpersist()
df_fact_benchmark.unpersist()
del df_benchmark_silver, df_fact, df_fact_benchmark
gc.collect()
print("✓ Đã giải phóng bộ nhớ")

#### Đổi Điểm chuẩn theo phương thức Điểm xét tốt nghiệp THPT thành Điểm chuẩn theo phương thức Điểm thi THPT

In [ ]:
# spark.sql("""UPDATE nessie.gold_tables.fact_benchmark
# SET selectionMethodKey = 2
# WHERE selectionMethodKey = 4;
# """)

### 3.2. Fact_Score_Distribution_By_Subject - Phân bố điểm theo môn

In [ ]:
print("Đang load Fact_Score_Distribution_By_Subject...")
import gc

# Đọc dữ liệu từ Silver
df_student_scores = spark.table("nessie.silver_tables.student_scores")

# Đọc dimension tables
df_dim_time_gold = spark.table("nessie.gold_tables.dim_time")
df_dim_region_gold = spark.table("nessie.gold_tables.dim_region")
df_dim_subject_gold = spark.table("nessie.gold_tables.dim_subject")

# Explode map scores thành từng dòng (subjectId - điểm)
# Vì scores giờ là Map<Int, Float> (subjectId -> score)
# subjectId ở đây là INTEGER, chính là cột subjectId từ bảng silver.subject
df_scores_exploded = df_student_scores.select(
    "regionId",
    "year",
    explode("scores").alias("subjectId", "score")  # subjectId = key của map (Int)
)

# Phân loại điểm vào các mốc điểm (0.2, 0.4, 0.6, ..., 9.8, 10.0)
# Làm tròn điểm đến bội số của 0.2 gần nhất
df_scores_exploded = df_scores_exploded.withColumn(
    "scoreThreshold",
    (floor(col("score") / 0.2) * 0.2).cast("float")
)

# Đếm số lượng học sinh theo từng môn, khu vực, năm và ngưỡng điểm
df_distribution = df_scores_exploded.groupBy(
    "regionId",
    "subjectId",  # Đây là subjectId (Int) từ scores map
    "year",
    "scoreThreshold"
).agg(
    count("*").alias("quantity")
)

# Giải phóng df_scores_exploded
df_scores_exploded.unpersist()
del df_scores_exploded

# Join với các dimension tables
# Join với dim_subject bằng subjectId để lấy subjectKey
df_fact = df_distribution \
    .join(df_dim_region_gold, df_distribution["regionId"] == df_dim_region_gold["regionId"], "left") \
    .join(df_dim_subject_gold, df_distribution["subjectId"] == df_dim_subject_gold["subjectId"], "left") \
    .join(df_dim_time_gold, df_distribution["year"] == df_dim_time_gold["year"], "left")

# Giải phóng dimension tables
df_dim_time_gold.unpersist()
df_dim_region_gold.unpersist()
df_dim_subject_gold.unpersist()

# Tạo SDBSKey tự động tăng
window_spec = Window.orderBy("timeKey", "regionKey", "subjectKey", "scoreThreshold")
df_fact = df_fact.withColumn("SDBSKey", row_number().over(window_spec))

# Select các cột cuối cùng
# Lưu ý: Lấy subjectKey từ df_dim_subject_gold (không phải subjectId)
df_fact_score_dist_subject = df_fact.select(
    "SDBSKey",
    "regionKey",
    "subjectKey",  # subjectKey từ dim_subject (surrogate key)
    "timeKey",
    col("scoreThreshold").cast("float"),
    col("quantity").cast("float")
)

# Ghi vào Gold layer
df_fact_score_dist_subject.writeTo("nessie.gold_tables.fact_score_distribution_by_subject") \
    .using("iceberg") \
    .partitionedBy("timeKey") \
    .createOrReplace()

print(f"Đã load {df_fact_score_dist_subject.count()} dòng vào fact_score_distribution_by_subject")
df_fact_score_dist_subject.show(10)

# Giải phóng bộ nhớ
df_student_scores.unpersist()
df_distribution.unpersist()
df_fact.unpersist()
df_fact_score_dist_subject.unpersist()
del df_student_scores, df_distribution, df_fact, df_fact_score_dist_subject, df_dim_time_gold, df_dim_region_gold, df_dim_subject_gold
gc.collect()
print("✓ Đã giải phóng bộ nhớ")

### 3.3. Fact_Score_Distribution_By_Subject_Group - Phân bố điểm theo khối thi

In [ ]:
print("Đang load Fact_Score_Distribution_By_Subject_Group...")
import gc

# Đọc dữ liệu từ Silver layer
df_student_scores = spark.table("nessie.silver_tables.student_scores")
df_subject_group = spark.table("nessie.silver_tables.subject_group")
df_subject = spark.table("nessie.silver_tables.subject")

# Đọc dimension tables từ Gold
df_dim_time_gold = spark.table("nessie.gold_tables.dim_time")
df_dim_region_gold = spark.table("nessie.gold_tables.dim_region")
df_dim_subject_group_gold = spark.table("nessie.gold_tables.dim_subject_group")

# Tạo mapping dictionaries
subject_name_to_id = {row['subjectName']: row['subjectId'] for row in df_subject.collect()}
subject_groups_dict = {}
for row in df_subject_group.collect():
    group_id = row['subjectGroupId']
    subjects = [s.strip() for s in row['subjectCombination'].split('-')]
    subject_groups_dict[group_id] = subjects

# Broadcast để worker nodes truy cập nhanh
subject_map_broadcast = spark.sparkContext.broadcast(subject_name_to_id)
subject_groups_broadcast = spark.sparkContext.broadcast(subject_groups_dict)

# UDF tính điểm tổng theo khối thi
def create_subject_group_scores_map(scores_map):
    if scores_map is None:
        return {}
    
    subject_name_to_id_map = subject_map_broadcast.value
    subject_groups = subject_groups_broadcast.value
    result = {}
    
    for group_id, required_subjects in subject_groups.items():
        total_score = 0.0
        has_all_subjects = True
        
        for subject_name in required_subjects:
            subject_id = subject_name_to_id_map.get(subject_name)
            if subject_id is None or subject_id not in scores_map:
                has_all_subjects = False
                break
            total_score += scores_map[subject_id]
        
        if has_all_subjects:
            result[group_id] = total_score
    
    return result

create_group_scores_udf = udf(create_subject_group_scores_map, MapType(StringType(), FloatType()))

# Tạo cột mới chứa mapping subjectGroupId -> totalScore
df_student_with_group_scores = df_student_scores.withColumn(
    "subject_group_scores",
    create_group_scores_udf(col("scores"))
)

# Explode map thành rows
df_scores_exploded = df_student_with_group_scores.select(
    "regionId",
    "year",
    explode("subject_group_scores").alias("subjectGroupId", "totalScore")
)

# Giải phóng df_student_with_group_scores
df_student_with_group_scores.unpersist()
del df_student_with_group_scores

# Phân loại điểm vào khoảng
df_scores_exploded = df_scores_exploded.withColumn(
    "scoreRange",
    floor(col("totalScore")).cast("float")
)

# Đếm số lượng học sinh
df_distribution = df_scores_exploded.groupBy(
    "regionId",
    "subjectGroupId",
    "year",
    "scoreRange"
).agg(
    count("*").alias("quantity")
)

# Giải phóng df_scores_exploded
df_scores_exploded.unpersist()
del df_scores_exploded

# Tạo mapping dictionaries từ dimension tables
region_id_to_key = {row['regionId']: row['regionKey'] for row in df_dim_region_gold.select("regionId", "regionKey").collect()}
subject_group_id_to_key = {str(row['subjectGroupId']).strip().upper(): row['subjectGroupKey'] for row in df_dim_subject_group_gold.select("subjectGroupId", "subjectGroupKey").collect()}
year_to_time_key = {row['year']: row['timeKey'] for row in df_dim_time_gold.select("year", "timeKey").collect()}

# Broadcast mappings
region_map_bc = spark.sparkContext.broadcast(region_id_to_key)
subject_group_map_bc = spark.sparkContext.broadcast(subject_group_id_to_key)
year_map_bc = spark.sparkContext.broadcast(year_to_time_key)

# UDF để map các natural keys sang surrogate keys
def map_to_keys(regionId, subjectGroupId, year):
    region_key = region_map_bc.value.get(regionId)
    normalized_group_id = str(subjectGroupId).strip().upper() if subjectGroupId is not None else None
    subject_group_key = subject_group_map_bc.value.get(normalized_group_id)
    time_key = year_map_bc.value.get(year)
    return (region_key, subject_group_key, time_key)

map_keys_udf = udf(map_to_keys, StructType([
    StructField("regionKey", IntegerType(), True),
    StructField("subjectGroupKey", IntegerType(), True),
    StructField("timeKey", IntegerType(), True)
]))

# Áp dụng mapping
df_with_keys = df_distribution.withColumn(
    "keys",
    map_keys_udf(col("regionId"), col("subjectGroupId"), col("year"))
)

# Extract keys từ struct
df_with_keys = df_with_keys.withColumn("regionKey", col("keys.regionKey")) \
    .withColumn("subjectGroupKey", col("keys.subjectGroupKey")) \
    .withColumn("timeKey", col("keys.timeKey")) \
    .drop("keys", "regionId", "subjectGroupId", "year")

# Tạo surrogate key
window_spec = Window.orderBy("timeKey", "regionKey", "subjectGroupKey", "scoreRange")
df_with_keys = df_with_keys.withColumn("SDBSGKey", row_number().over(window_spec))

# Select các cột cuối cùng
df_fact_score_dist_group = df_with_keys.select(
    "SDBSGKey",
    "regionKey",
    "subjectGroupKey",
    "timeKey",
    col("scoreRange").cast("float"),
    col("quantity").cast("float")
)

# Ghi vào Gold layer
df_fact_score_dist_group.writeTo("nessie.gold_tables.fact_score_distribution_by_subject_group") \
    .using("iceberg") \
    .partitionedBy("timeKey") \
    .createOrReplace()

print(f"Đã load {df_fact_score_dist_group.count()} dòng vào fact_score_distribution_by_subject_group")
df_fact_score_dist_group.show(10)

# Giải phóng bộ nhớ
df_student_scores.unpersist()
df_subject_group.unpersist()
df_subject.unpersist()
df_dim_time_gold.unpersist()
df_dim_region_gold.unpersist()
df_dim_subject_group_gold.unpersist()
df_distribution.unpersist()
df_with_keys.unpersist()
df_fact_score_dist_group.unpersist()
del df_student_scores, df_subject_group, df_subject, df_dim_time_gold, df_dim_region_gold, df_dim_subject_group_gold
del df_distribution, df_with_keys, df_fact_score_dist_group
gc.collect()
print("✓ Đã giải phóng bộ nhớ")

## 4. Dừng Spark Session

In [ ]:
# Giải phóng bộ nhớ trước khi dừng Spark
import gc

# Xóa tất cả cache
spark.catalog.clearCache()

# Thu gom rác
gc.collect()
print("✓ Đã giải phóng toàn bộ bộ nhớ")

# Dừng Spark Session để giải phóng resources
spark.stop()
print("✓ Spark Session đã được dừng!")